# Importing important libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization, Input, GlobalAveragePooling2D
from keras.utils.vis_utils import plot_model
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.applications import InceptionResNetV2


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
train = pd.read_csv("/kaggle/input/cassava-leaf-disease-classification/train.csv")
train['label'] = train['label'].astype('string')
train.head()

In [ ]:
diseases = pd.read_json("/kaggle/input/cassava-leaf-disease-classification/label_num_to_disease_map.json", typ='series')
diseases

In [ ]:
train['label'].value_counts(normalize=True)

In [ ]:
IMG_SIZE = 320

# Data Augmentation

In [ ]:
datagen = image.ImageDataGenerator(rotation_range=360, width_shift_range=0.1,
                                   height_shift_range=0.1, brightness_range=[0.2,1.5],
                                   shear_range=25, zoom_range=0.3,
                                   channel_shift_range=0.1, horizontal_flip=True,
                                   vertical_flip=True, rescale=1/255,
                                   validation_split=0.15)

val_datagen = image.ImageDataGenerator(rescale=1/255, validation_split = 0.15)

In [ ]:
train_generator = datagen.flow_from_dataframe(
    dataframe=train,
    directory="/kaggle/input/cassava-leaf-disease-classification/train_images",
    x_col='image_id',
    y_col='label',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32,
    subset='training',
    shuffle = True,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=train,
    directory="/kaggle/input/cassava-leaf-disease-classification/train_images",
    x_col='image_id',
    y_col='label',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32,
    subset='validation',
    class_mode = 'categorical',
    shuffle = True
)

In [ ]:
train_imgs, labels = next(train_generator)
print(train_imgs.shape)

In [ ]:
plt.figure(figsize=(20,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_imgs[i])
    label1 = np.argmax(labels[i])
    plt.xlabel(diseases.get(label1))
plt.show()

# Model

In [ ]:
model = tf.keras.Sequential()
model.add(InceptionResNetV2(include_top=False,
                            weights='../input/keras-pretrained-models/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5',
                            input_shape=[IMG_SIZE,IMG_SIZE,3]))
model.add(BatchNormalization(axis=-1))
model.add(GlobalAveragePooling2D())
model.add(Dense(5, activation='softmax'))

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adamax(learning_rate=0.01),
              metrics=['accuracy'])
model.summary()

In [ ]:
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

In [ ]:
model_save = ModelCheckpoint("Model", 
                             save_best_only = True, 
                             save_weights_only = True,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)
early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, 
                           patience = 5, mode = 'min', verbose = 1,
                           restore_best_weights = True)
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.3, 
                              patience = 2, min_delta = 0.001, 
                              mode = 'min', verbose = 1)

# Training the Model

In [ ]:
history = model.fit_generator(train_generator, steps_per_epoch=17118//32,
                              epochs=25, validation_data=val_generator,
                              validation_steps = 4279//32, verbose = 1,
                              callbacks = [model_save, early_stop, reduce_lr])

In [ ]:
model_history = history.history
loss_accuracy_train = model.evaluate(train_generator)
print("Training Loss: {:.4f}".format(loss_accuracy_train[0]))
print("Training Accuracy: {:.2%}".format(loss_accuracy_train[1]))
loss_accuracy = model.evaluate(val_generator)
print("Validation Loss: {:.4f}".format(loss_accuracy[0]))
print("Validation Accuracy: {:.2%}".format(loss_accuracy[1]))

In [ ]:
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
fig.set_size_inches(12,4)

ax[0].plot(history.history['accuracy'])
ax[0].plot(history.history['val_accuracy'])
ax[0].set_title('Training Accuracy vs Validation Accuracy')
ax[0].set_ylabel('Accuracy')
ax[0].set_xlabel('Epoch')
ax[0].legend(['Train', 'Validation'], loc='upper left')

ax[1].plot(history.history['loss'])
ax[1].plot(history.history['val_loss'])
ax[1].set_title('Training Loss vs Validation Loss')
ax[1].set_ylabel('Loss')
ax[1].set_xlabel('Epoch')
ax[1].legend(['Train', 'Validation'], loc='upper left')

plt.show()

# Prediction

In [ ]:
preds = []
ss = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')

for image in ss.image_id:
    img = tf.keras.preprocessing.image.load_img('../input/cassava-leaf-disease-classification/test_images/' + image)
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.keras.preprocessing.image.smart_resize(img, (IMG_SIZE, IMG_SIZE))
    img = tf.reshape(img, (-1, IMG_SIZE, IMG_SIZE, 3))
    prediction = model.predict(img/255)
    preds.append(np.argmax(prediction))

submission = pd.DataFrame({'image_id': ss.image_id, 'label': preds})
submission

In [ ]:
submission.to_csv('submission.csv', index = False)